In [1]:
import pandas as pd
import os
import gc
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge
from sklearn.preprocessing import MinMaxScaler
import math
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('data/train.csv')
testA = pd.read_csv('data/testA.csv')


In [3]:
train.head()


,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,...,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
0,0,35000.0,5,19.52,917.97,E,E2,320.0,2 years,2,...,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0
1,1,18000.0,5,18.49,461.90,D,D2,219843.0,5 years,0,...,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN
2,2,12000.0,5,16.99,298.17,D,D3,31698.0,8 years,0,...,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0
3,3,11000.0,3,7.26,340.96,A,A4,46854.0,10+ years,1,...,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0
4,4,3000.0,3,12.99,101.07,C,C2,54.0,NaN,1,...,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0


In [4]:
data = pd.concat([train, testA], axis=0, ignore_index=True)


In [5]:
print(sorted(data['grade'].unique()))
print(sorted(data['subGrade'].unique()))

['A', 'B', 'C', 'D', 'E', 'F', 'G']
['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G1', 'G2', 'G3', 'G4', 'G5']


In [6]:
data['employmentLength'].value_counts(dropna=False).sort_index()

employmentLength
1 year        65671
10+ years    328525
2 years       90565
3 years       80163
4 years       59818
5 years       62645
6 years       46582
7 years       44230
8 years       45168
9 years       37866
< 1 year      80226
NaN           58541
Name: count, dtype: int64

In [7]:
data['employmentLength'].replace(to_replace='10+ years', value='10 years', inplace=True)
data['employmentLength'].replace('< 1 year', '0 years', inplace=True)
from utils_jessie import employmentLength_to_int
data['employmentLength'] = data['employmentLength'].apply(employmentLength_to_int)


In [8]:
data['employmentLength'].value_counts(dropna=False).sort_index()


employmentLength
0.0      80226
1.0      65671
2.0      90565
3.0      80163
4.0      59818
5.0      62645
6.0      46582
7.0      44230
8.0      45168
9.0      37866
10.0    328525
NaN      58541
Name: count, dtype: int64

In [9]:
data['earliesCreditLine'].sample(5)


381325    Sep-1987
255622    Jan-1978
199521    Sep-2003
322671    Mar-2003
568623    Oct-2002
Name: earliesCreditLine, dtype: object

In [10]:
data['earliesCreditLine'] = data['earliesCreditLine'].apply(lambda s: int(s[-4:]))


In [11]:
data['earliesCreditLine'].describe()

count    1000000.000000
mean        1998.688632
std            7.606231
min         1944.000000
25%         1995.000000
50%         2000.000000
75%         2004.000000
max         2015.000000
Name: earliesCreditLine, dtype: float64

In [12]:
data.head()


,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,...,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
0,0,35000.0,5,19.52,917.97,E,E2,320.0,2.0,2,...,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0
1,1,18000.0,5,18.49,461.90,D,D2,219843.0,5.0,0,...,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN
2,2,12000.0,5,16.99,298.17,D,D3,31698.0,8.0,0,...,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0
3,3,11000.0,3,7.26,340.96,A,A4,46854.0,10.0,1,...,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0
4,4,3000.0,3,12.99,101.07,C,C2,54.0,NaN,1,...,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0


In [13]:
# 部分类别特征
cate_features = ['grade', 'subGrade', 'employmentTitle', 'homeOwnership', 'verificationStatus', 'purpose', 'postCode', 'regionCode', \
                 'applicationType', 'initialListStatus', 'title', 'policyCode']
for f in cate_features:
    print(f, '类型数：', data[f].nunique())

grade 类型数： 7
subGrade 类型数： 35
employmentTitle 类型数： 298101
homeOwnership 类型数： 6
verificationStatus 类型数： 3
purpose 类型数： 14
postCode 类型数： 935
regionCode 类型数： 51
applicationType 类型数： 2
initialListStatus 类型数： 2
title 类型数： 47903
policyCode 类型数： 1


In [14]:
# 类型数在2之上，又不是高维稀疏的
data = pd.get_dummies(data, columns=['grade', 'subGrade', 'homeOwnership', 'verificationStatus', 'purpose', 'regionCode'], drop_first=True)

In [15]:
# 处理一下缺失值
# numerical_fea 是一个列表，包含 data_train 中所有非 object 类型的列名（即数值型列）
numerical_fea  = list(data.select_dtypes(exclude=['object']).columns)
# 使用 filter 函数和 lambda 表达式来筛选出那些不在 numerical_fea 列表中的列名。
category_fea = list(filter(lambda x: x not in numerical_fea, list(data.columns)))
label = 'isDefault'
numerical_fea.remove(label)

# 按照中位数填充数值型特征
data[numerical_fea] = data[numerical_fea].fillna( data[numerical_fea].median() )

#按照众数填充类别型特征
data[category_fea] = data[category_fea].fillna(data[category_fea].mode())

In [16]:
#查看缺失值情况
data.isnull().sum()

id               0
loanAmnt         0
term             0
interestRate     0
installment      0
                ..
regionCode_46    0
regionCode_47    0
regionCode_48    0
regionCode_49    0
regionCode_50    0
Length: 151, dtype: int64

In [18]:
# 高维类别特征需要进行转换
for f in ['employmentTitle', 'postCode', 'title']:
    data[f+'_cnts'] = data.groupby([f])['id'].transform('count')
    data[f+'_rank'] = data.groupby([f])['id'].rank(ascending=False).astype(int)
    del data[f]

In [19]:
data.head(5)

,id,loanAmnt,term,interestRate,installment,employmentLength,annualIncome,issueDate,isDefault,dti,...,regionCode_47,regionCode_48,regionCode_49,regionCode_50,employmentTitle_cnts,employmentTitle_rank,postCode_cnts,postCode_rank,title_cnts,title_rank
0,0,35000.0,5,19.52,917.97,2.0,110000.0,2014-07-01,1.0,17.05,...,False,False,False,False,1392,1392,2646,2646,8688,8688
1,1,18000.0,5,18.49,461.90,5.0,46000.0,2012-08-01,0.0,27.83,...,False,False,False,False,151,151,4751,4751,37,37
2,2,12000.0,5,16.99,298.17,8.0,74000.0,2015-10-01,0.0,22.77,...,False,False,False,False,2,2,2167,2167,491400,491400
3,3,11000.0,3,7.26,340.96,10.0,118000.0,2015-08-01,0.0,17.21,...,False,False,False,False,2,2,689,689,185386,185386
4,4,3000.0,3,12.99,101.07,6.0,29000.0,2016-03-01,0.0,32.16,...,False,False,False,False,63978,63978,2161,2161,5896,5896


In [20]:
# 保存文件
import pyarrow as pa
import pyarrow.csv as pc
table = pa.Table.from_pandas(data)
pc.write_csv(table, "data_for_model.csv")